In [ ]:
import argparse
import logging
import os
import sys

from redbaron import RedBaron

from toolz.curried import keyfilter

# Loading

In [ ]:
source_dir = '../../openfisca-france/openfisca_france/model/'
filenames = []

for root, directories, files in os.walk(source_dir):    
    for filename in files:
        complete_filename = os.path.join(root, filename)
        assert complete_filename[:len(source_dir)] == source_dir
        complete_filename = complete_filename[len(source_dir):]
        filenames.append(complete_filename)
filenames

In [ ]:
filenames.remove('base.py')
filenames.remove('datatrees.py')
filenames.remove('prelevements_obligatoires/prelevements_sociaux/cotisations_sociales/preprocessing.py')

In [ ]:
redbaron_trees = {}
for filename in filenames:
    with open(source_dir + filename) as source_file:
        source_code = source_file.read()
    red = RedBaron(source_code)
    redbaron_trees[filename] = red
    print('{} parsed'.format(filename))

# Custom exceptions

In [ ]:
angry_rbnode = None
angry_state = None

In [ ]:
class ParsingException(Exception):
    def __init__(self, message, rbnode, s):
        global angry_rbnode
        global angry_state

        angry_rbnode = rbnode
        angry_state = s

        super(ParsingException, self).__init__(message)

In [ ]:
class NotImplementedParsingError(ParsingException):
    pass

In [ ]:
class AssertionParsingError(ParsingException):
    pass

In [ ]:
def my_assert(cond, rbnode, s):
    if cond:
        return
    
    raise AssertionParsingError('', rbnode, s)

# Helpers

In [ ]:
import unicodedata

def rbnode_to_exception(rbnode):
    '''Because exceptions are ASCII only in python2'''
    str1 = rbnode.dumps() # unicode string wrongly known as 'str'
    str2 = unicode(str1, 'utf-8') # unicode string as unicode
    str3 = unicodedata.normalize('NFKD', str2).encode('ascii', 'ignore') # ignore special chars
    
    return str3

In [ ]:
def parse_date(atomtrailer, s):
    my_assert(atomtrailer.type == 'atomtrailers', atomtrailer, s)
    my_assert(len(atomtrailer.value) == 2, atomtrailer, s)
    my_assert(atomtrailer.value[0].type == 'name', atomtrailer, s)
    my_assert(atomtrailer.value[0].value == 'date', atomtrailer, s)
    call_node = atomtrailer.value[1]
    my_assert(call_node.type == 'call', atomtrailer, s)
    my_assert(len(call_node.value) == 3, atomtrailer, s)
    my_assert(call_node.value[0].type == 'call_argument', atomtrailer, s)
    my_assert(not call_node.value[0].target, atomtrailer, s)
    my_assert(call_node.value[0].value.type == 'int', atomtrailer, s)
    year = call_node.value[0].value.value
    my_assert(call_node.value[1].type == 'call_argument', atomtrailer, s)
    my_assert(not call_node.value[1].target, atomtrailer, s)
    my_assert(call_node.value[1].value.type == 'int', atomtrailer, s)
    month = call_node.value[1].value.value
    my_assert(call_node.value[2].type == 'call_argument', atomtrailer, s)
    my_assert(not call_node.value[2].target, atomtrailer, s)
    my_assert(call_node.value[2].value.type == 'int', atomtrailer, s)
    day = call_node.value[2].value.value
    
    return {'year': year, 'month': month, 'day': day}

In [ ]:
def parse_enum(atomtrailers, s):
    my_assert(atomtrailers.type == 'atomtrailers', rbnode, s)
    
    my_assert(len(atomtrailers.value) == 2, rbnode, s)
    my_assert(atomtrailers.value[0].type == 'name', rbnode, s)
    my_assert(atomtrailers.value[0].value == 'Enum', rbnode, s)
    
    call_node = atomtrailers.value[1]
    my_assert(call_node.type == 'call', rbnode, s)
    my_assert(len(call_node.value) == 1, rbnode, s)
    my_assert(call_node.value[0].type == 'call_argument', rbnode, s)
    my_assert(not call_node.value[0].target, rbnode, s)
    
    enum_list_node = call_node.value[0].value
    my_assert(enum_list_node.type == 'list', rbnode, s)
    
    enum_list = []
    for element in enum_list_node.value:
        my_assert(element.type == 'unicode_string', rbnode, s)
        enum_list.append(element.value)
        
    return enum_list

In [ ]:
def parse_parameter_path(atomtrailers, first_index, s):
    my_assert(atomtrailers.type == 'atomtrailers', atomtrailers, s)
    
    parameter_path = []
    for i in range(first_index, len(atomtrailers.value)):
        path_component = atomtrailers.value[i]
        my_assert(path_component.type == 'name', rbnode, s)
        parameter_path.append(path_component.value)

    return parameter_path

In [ ]:
def parse_arguments(call, global_index, local_index):
    my_assert(call.type == 'call', rbnode, s)
    
    arg_list = []
    arg_dict = {}
    
    no_target = True
    args = call.value
    for arg in args:
        my_assert(args.type == 'call_argument', rbnode, s)
        if args[0].target:
            no_target = False
            my_assert(arg.type == 'call_argument', rbnode, s)
            my_assert(arg.target.type == 'name', rbnode, s)
            target = arg.target.value
        else:
            my_assert(no_target, rbnode, s)
            my_assert(arg.value.type == 'name', rbnode, s)

        child_state = {
            'keyword': 'expression',
            'local_variables': s['local_variables'],
        }
        visit_function_rbnode(arg.value, child_state)
        value = child_state['tmp_var']
        
        if args[0].target:
            arg_dict[target] = value
        else:
            arg_list.append(value)

        return arg_list, arg_dict

# Module traversal functions

In [ ]:
def visit_module_rbnode(rbnode, s):
    visitors = keyfilter(lambda key: key.startswith('visit_module_'), globals()) # should be defined once
    visitor = visitors.get('visit_module_' + rbnode.type)
    if visitor is None:
        raise NotImplementedParsingError(
            'Module visitor not declared for type="{type}"'.format(
                type=rbnode.type,
                ), rbnode, s)
    ofnode = visitor(rbnode, s)
    return ofnode


In [ ]:
def visit_module_endl(rbnode, s):
    return

In [ ]:
def visit_module_from_import(rbnode, s):
    my_assert(s['keyword'] == 'module', rbnode, s)
    
    # unmodified (TODO)
    s['imports'].append(rbnode)

In [ ]:
def visit_module_import(rbnode, s):
    my_assert(s['keyword'] == 'module', rbnode, s)
    
    # unmodified (TODO)
    s['imports'].append(rbnode)

In [ ]:
def visit_module_comment(rbnode, s):
    # comments are discarded for the moment (TODO)
    return

In [ ]:
def visit_module_class(rbnode, s):
    my_assert(s['keyword'] == 'module', rbnode, s)
    
    name = rbnode.name
    
    if name == 'rsa_ressource_calculator':
        return
    
    my_assert(not rbnode.decorators, rbnode, s)
    
    upper_classes = []
    for upper_class in rbnode.inherit_from:
        my_assert(upper_class.type == 'name', rbnode, s)
        upper_classes.append(upper_class.value)
        
    class_obj = {
        'type': 'class',
        'name': name,
        'upper_classes': upper_classes,
        'content': rbnode.value,
        }

    s['classes'].append(class_obj)

In [ ]:
def visit_module_def(rbnode, s):
    my_assert(s['keyword'] == 'module', rbnode, s)
    
    if rbnode.name in ['_revprim', 'preload_zone_apl']:
        return
    
    # unmodified (TODO)
    s['auxiliary_functions'].append(rbnode)

In [ ]:
def visit_module_assignment(rbnode, s):
    my_assert(s['keyword'] == 'module', rbnode, s)

    my_assert(rbnode.operator == '', rbnode, s)
    
    my_assert(rbnode.target.type == 'name', rbnode, s)
    name = rbnode.target.value
    
    if name in ['zone_apl_by_depcom']:
        return
    
    if rbnode.value.type == 'int':
        s['constants'].append({
                'name': name,
                'type': 'int',
                'value': rbnode.value.value,
            })
        return
    
    if rbnode.value.type == 'name':
        my_assert(rbnode.value.value == 'None', rbnode, s)
        s['constants'].append({
                'name': name,
                'type': 'None',
                'value': None,
            })
        return
    
    if rbnode.value.type == 'atomtrailers':
        atomtrailers = rbnode.value

        my_assert(atomtrailers.value[0].type == 'name', rbnode, s)
        function_name = atomtrailers.value[0].value
        if function_name == 'Enum':
            enum_list = parse_enum(atomtrailers, s)

            s['enums'].append({
                'name': name,
                'enum_list': enum_list,
            })
            return

        if function_name == 'logging':
            # ignore logging
            return

        raise ParsingException('Unknown atomtrailers', rbnode, s)

    raise ParsingException('Unknown type', rbnode, s)


# Module parsing

In [ ]:
parsed_modules = {}

for name in filenames:
    print('Visiting ' + name)
    red = redbaron_trees[name]
    

    
    s = {
        'keyword': 'module',
        'module_name': name,
        'imports': [],
        'classes': [],
        'enums': [],
        'auxiliary_functions': [],
        'constants': [],
        }
    
    for rbnode in red:
        visit_module_rbnode(rbnode, s)
        
    parsed_modules[name] = {
        'imports': s['imports'],
        'classes': s['classes'],
        'enums': s['enums'],
        'auxiliary_functions': s['auxiliary_functions'],
        'constants': s['constants'],
    }

# Class traversal functions

In [ ]:
def visit_class_rbnode(rbnode, s):
    visitors = keyfilter(lambda key: key.startswith('visit_class_'), globals()) # should be defined once
    visitor = visitors.get('visit_class_' + rbnode.type)
    if visitor is None:
        raise NotImplementedParsingError(
            'Class visitor not declared for type="{type}"'.format(
                type=rbnode.type,
                ), rbnode, s)
    ofnode = visitor(rbnode, s)
    return ofnode


In [ ]:
def visit_class_endl(rbnode, s):
    return

In [ ]:
def visit_class_assignment(rbnode, s):
    my_assert(s['keyword'] == 'class', rbnode, s)
    
    my_assert(rbnode.operator == '', rbnode, s)
    
    my_assert(rbnode.target.type == 'name', rbnode, s)
    target = rbnode.target.value
    
    if target == 'column':
        if rbnode.value.type == 'atomtrailers':
            my_assert(len(rbnode.value.value) == 2, rbnode, s)

            my_assert(rbnode.value.value[0].type == 'name', rbnode, s)
            column_name = rbnode.value.value[0].value

            call_node = rbnode.value.value[1]
            my_assert(call_node.type == 'call', rbnode, s)
            column_args = {}
            for arg in call_node.value:
                my_assert(arg.target.type == 'name', rbnode, s)
                column_args[arg.target.value] = arg.value

            my_assert('column' not in s['class_variables'].keys(), rbnode, s)
            s['class_variables']['column'] = column_name
            s['class_variables']['column_args'] = column_args

        elif rbnode.value.type == 'name':
            column_name = rbnode.value.value

            my_assert('column' not in s['class_variables'].keys(), rbnode, s)
            s['class_variables']['column'] = column_name
        else:
            raise NotImplementedParsingError('Unknown type', rbnode, s)
    
    elif target == 'entity_class':
        my_assert(rbnode.value.type == 'name', rbnode, s)
        
        my_assert('entity_class' not in s, rbnode, s)
        s['entity_class'] = rbnode.value.value
              
    elif target == 'label':
        # can be unicode_string or string_chain ! (TODO)
        # my_assert(rbnode.value.type == 'unicode_string', rbnode, s)
        
        my_assert('label' not in s, rbnode, s)
        s['label'] = rbnode.value
        
    elif target == 'start_date':
        date = parse_date(rbnode.value, s)
        
        my_assert('start_date' not in s, rbnode, s)
        s['start_date'] = date
         
    elif target == 'stop_date':
        date = parse_date(rbnode.value, s)

        my_assert('stop_date' not in s, rbnode, s)
        s['stop_date'] = date
        
    elif target == 'url':
        # can be a tuple, see revnet (TODO)
        # my_assert(rbnode.value.type in ['string', 'unicode_string'], rbnode, s)
        
        my_assert('url' not in s, rbnode, s)
        s['url'] = rbnode.value.value
        
             
    elif target == 'operation':
        my_assert(rbnode.value.type == 'string', rbnode, s)
        
        my_assert('operation' not in s, rbnode, s)
        s['operation'] = rbnode.value.value
        
             
    elif target == 'variable':
        my_assert(rbnode.value.type == 'name', rbnode, s)
        
        my_assert('variable' not in s, rbnode, s)
        s['variable'] = rbnode.value.value
              
    elif target == 'cerfa_field':
        # my_assert(rbnode.value.type == 'unicode_string', rbnode, s)
        # can be a unicode string or a dict
        
        my_assert('cerfa_field' not in s, rbnode, s)
        s['cerfa_field'] = rbnode.value
                
    elif target == 'is_permanent':
        my_assert(rbnode.value.type == 'name', rbnode, s)
        my_assert(rbnode.value.value in ['True', 'False'], rbnode, s)        
        
        my_assert('is_permanent' not in s, rbnode, s)
        s['is_permanent'] = rbnode.value.value == 'True'
    
    elif target == 'base_function':
        my_assert(rbnode.value.type == 'name', rbnode, s)
        
        my_assert('base_function' not in s, rbnode, s)
        s['base_function'] = rbnode.value.value
              
    elif target == 'calculate_output':
        my_assert(rbnode.value.type == 'name', rbnode, s)
        
        my_assert('calculate_output' not in s, rbnode, s)
        s['calculate_output'] = rbnode.value.value
              
    elif target == 'set_input':
        my_assert(rbnode.value.type == 'name', rbnode, s)
        
        my_assert('set_input' not in s, rbnode, s)
        s['set_input'] = rbnode.value.value
        
    elif target == 'role':
        my_assert(rbnode.value.type == 'name', rbnode, s)
        
        my_assert('role' not in s, rbnode, s)
        s['role'] = rbnode.value.value
              

    else:            
        raise NotImplementedParsingError('Unknown class variable {}'.format(target), rbnode, s)
            


In [ ]:
def visit_class_def(rbnode, s):
    my_assert(s['keyword'] == 'class', rbnode, s)
    name = rbnode.name
    
    decorators = rbnode.decorators

    arguments = []
    for arg in rbnode.arguments:
        my_assert(arg.type == 'def_argument', rbnode, s)
        my_assert(arg.target.type == 'name', rbnode, s)
        arguments.append(arg.target.value)
        my_assert(not arg.value, rbnode, s)
    
    instructions = rbnode.value # unmodified (TODO)
    
    my_assert(name not in s['class_functions'], rbnode, s)
    s['class_functions'][name] = {
        'arguments': arguments,
        'decorators': decorators,
        'instructions': instructions,
    }

In [ ]:
def visit_class_comment(rbnode, s):
    # ignored (TODO)
    return

In [ ]:
def visit_class_string(rbnode, s):
    # ignored (TODO)
    return

# Class parsing

In [ ]:
parsed_classes = {}

for module_name, module in parsed_modules.items():
    print('Visiting module {} to parse its classes.'.format(module_name))
    
    parsed_classes[module_name] = {
        'parsed_classes': {},
    }
    
    for cl in module['classes']:
        class_name = cl['name']
        print('Visiting class {}'.format(class_name))
        
        s = {
            'keyword': 'class',
            'class_name': name,
            'class_variables': {},
            'class_functions': {},
            }
    
        for rbnode in cl['content']:
            visit_class_rbnode(rbnode, s)

        parsed_classes[module_name]['parsed_classes'][class_name] = {
            'class_variables': s['class_variables'],
            'class_functions': s['class_functions'],
            }

# Function traversal visitors

In [ ]:
def visit_function_rbnode(rbnode, s):
    visitors = keyfilter(lambda key: key.startswith('visit_function_'), globals()) # should be defined once
    visitor = visitors.get('visit_function_' + rbnode.type)
    if visitor is None:
        raise NotImplementedParsingError(
            'Function visitor not declared for type="{type}"'.format(
                type=rbnode.type,
                ), rbnode, s)
    ofnode = visitor(rbnode, s)
    return ofnode


In [ ]:
def visit_function_endl(rbnode, s):
    return

In [ ]:
def visit_function_assignment(rbnode, s):
    my_assert(s['keyword'] == 'function', rbnode, s)
    
    my_assert(rbnode.target.type == 'name', rbnode, s)
    name = rbnode.target.value
    
    rbvalue = rbnode.value
    s['keyword'] = 'expression'
    visit_function_rbnode(rbvalue, s)
    
    s['local_variables'][name] = s['tmp_var']
    del s['tmp_var']
    s['keyword'] = 'function'

In [ ]:
def visit_function_atomtrailers(rbnode, s):
    my_assert(s['keyword'] == 'expression', rbnode, s)

    rb_first_item = rbnode.value[0]
    
    child_state = {
        'keyword': 'expression',
        'local_variables': s['local_variables'],
    }
    visit_function_rbnode(rb_first_item, child_state)
    first_item = child_state['tmp_var']

    if first_item['type'] == 'period':
        my_assert(len(rbnode.value) == 2, rbnode, s)

        my_assert(rbnode.value[1].type == 'name', rbnode, s)
        period_op = rbnode.value[1].value
        if period_op in ['this_month', 'n_2']:
            tmp_var = {
                'type': 'period', 
                'nodetype': 'period-operation', 
                'operator': period_op, 
                'operands': [first_item],
            }
            s['tmp_var'] = tmp_var
            return

        if period_op == 'start':
            tmp_var = {
                'type': 'instant', 
                'nodetype': 'period-to-instant', 
                'operator': 'start', 
                'operands': [first_item],
            }
            s['tmp_var'] = tmp_var
            return

        raise NotImplementedParsingError('Unknown period operand', rbnode, s)
            
    if first_item['type'] == 'simulation':

        my_assert(rbnode.value[1].type == 'name', rbnode, s)
        simulation_op = rbnode.value[1].value
        if simulation_op in ['calculate', 'compute', 'calculate_add']:
            my_assert(len(rbnode.value) == 3, rbnode, s)

            my_assert(rbnode.value[2].type == 'call', rbnode, s)
            args = rbnode.value[2].value
            my_assert(len(args) in [1, 2], rbnode, s)

            my_assert(args[0].type == 'call_argument', rbnode, s)
            my_assert(not args[0].target, rbnode, s)
            my_assert(args[0].value.type == 'string', rbnode, s)
            called_var = args[0].value.value
            operands = [called_var]

            if len(args) == 2:
                my_assert(args[1].type == 'call_argument', rbnode, s)
                my_assert(not args[1].target, rbnode, s)
                rb_arg_period = args[1].value
                child_state = {
                    'keyword': 'expression',
                    'local_variables': s['local_variables'],
                }
                visit_function_rbnode(rb_arg_period, child_state)
                arg_period = child_state['tmp_var']
                my_assert(arg_period['type'] == 'period', rbnode, s)
                operands.append(arg_period)

            tmp_var = {
                'type': 'value', 
                'nodetype': 'variable_for_period', 
                'operator': simulation_op, 
                'operands': operands,
            }
            s['tmp_var'] = tmp_var
            return

        if simulation_op == 'legislation_at':
            my_assert(rbnode.value[2].type == 'call', rbnode, s)
            args = rbnode.value[2].value
            my_assert(len(args) == 1, rbnode, s)
            rb_instant_arg = args[0]
            my_assert(rb_instant_arg.type == 'call_argument', rbnode, s)
            my_assert(not rb_instant_arg.target, rbnode, s)

            child_state = {
                'keyword': 'expression',
                'local_variables': s['local_variables'],
            }
            visit_function_rbnode(rb_instant_arg.value, child_state)
            instant_arg = child_state['tmp_var']
            my_assert(instant_arg['type'] == 'instant', rbnode, s)

            parameter_path = parse_parameter_path(rbnode, 3, s)

            tmp_var = {
                'type': 'parameter', 
                'nodetype': 'parameter', 
                'instant': instant_arg, 
                'path': parameter_path,
            }
            s['tmp_var'] = tmp_var
            return

        raise NotImplementedParsingError('Unknown simulation op.', rbnode, s)
            
    if first_item['type'] == 'self':

        my_assert(rbnode.value[1].type == 'name', rbnode, s)
        self_op = rbnode.value[1].value
        if self_op in ['split_by_roles', 'sum_by_entity']:
            my_assert(len(rbnode.value) == 3, rbnode, s)
            
            child_state = {
                'keyword': 'expression',
                'local_variables': s['local_variables'],
            }
            visit_function_rbnode(rbnode.value[2], child_state)
            args = child_state['tmp_var']

            tmp_var = {
                'type': 'value', 
                'nodetype': 'self_operation', 
                'operator': self_op, 
                'operands': args,
            }
            s['tmp_var'] = tmp_var
            return

        raise NotImplementedParsingError('Unknown self op.', rbnode, s)
            
    if first_item['type'] == 'parameter':
        parameter_path = parse_parameter_path(rbnode, 1, s)

        tmp_var = {
            'type': 'value', 
            'nodetype': 'parameter', 
            'instant': first_item['instant'], 
            'path': first_item['path'] + parameter_path,
        }
        s['tmp_var'] = tmp_var
        return
    
    if first_item['type'] == 'arithmetic_operation_tmp':
    
        if first_item['op'] in ['round', 'sum', 'not_']:
            my_assert(len(rbnode.value) == 2, rbnode, s)
            my_assert(rbnode.value[1].type == 'call', rbnode, s)
            args = rbnode.value[1].value
            
            if (len(args) == 1) and args[0].type == 'argument_generator_comprehension':
                # (TODO)
                tmp_var = {
                    'type': 'value', 
                    'nodetype': 'arithmetic_operation_comprehension',
                    'op': first_item['op'],
                    'rb_argument_generator_comprehension': args[0],
                }
                s['tmp_var'] = tmp_var
                return
                
            parsed_args = []
            for arg in args:
                my_assert(arg.type == 'call_argument', rbnode, s)
                my_assert(not arg.target, rbnode, s)
                
                child_state = {
                    'keyword': 'expression',
                    'local_variables': s['local_variables'],
                }
                visit_function_rbnode(arg.value, child_state)
                parsed_arg = child_state['tmp_var']
                parsed_args.append(parsed_arg)

            tmp_var = {
                'type': 'value', 
                'nodetype': 'arithmetic_operation', 
                'op': first_item['op'], 
                'operands': parsed_args,
            }
            s['tmp_var'] = tmp_var
            return
                   
        raise NotImplementedParsingError('Unknown arithmetic_operation_tmp.', rbnode, s)
        
    if first_item['type'] == 'apply_thresholds_tmp':
        tmp_var = {
            'type': 'value', 
            'nodetype': 'apply_thresholds', 
            'rbnode': rbnode,
        }
        s['tmp_var'] = tmp_var
        return

    raise NotImplementedParsingError('Unknown first item of an atomtrailers.', rbnode, s)
    

In [ ]:
def visit_function_binary_operator(rbnode, s):
    op = rbnode.value
    
    parsed_args = []
    for arg in [rbnode.first, rbnode.second]:
        child_state = {
            'keyword': 'expression',
            'local_variables': s['local_variables'],
        }
        visit_function_rbnode(arg, child_state)
        parsed_arg = child_state['tmp_var']
        parsed_args.append(parsed_arg)

    tmp_var = {
        'type': 'value', 
        'nodetype': 'arithmetic_operation', 
        'op': op, 
        'operands': parsed_args,
    }
    s['tmp_var'] = tmp_var
    return


In [ ]:
def visit_function_name(rbnode, s):
    name = rbnode.value
    
    if name in s['local_variables']:
        s['tmp_var'] = s['local_variables'][name]
        return

    if name in ['round', 'sum', 'not_']:
        tmp_var = {
            'type': 'arithmetic_operation_tmp', 
            'nodetype': 'arithmetic_operation_tmp', 
            'op': name, 
        }
        s['tmp_var'] = tmp_var
        return
    
    if name == 'apply_thresholds':
        # to deal with specifically (TODO)
        tmp_var = {
            'type': 'apply_thresholds_tmp', 
            'nodetype': 'apply_thresholds', 
        }
        s['tmp_var'] = tmp_var
        return
            
    raise NotImplementedParsingError('Unknown name {}'.format(name), rbnode, s)


In [ ]:
def visit_function_int(rbnode, s):
    tmp_var = {
        'type': 'value', 
        'nodetype': 'int', 
        'value': rbnode.value, 
    }
    s['tmp_var'] = tmp_var
    return


In [ ]:
def visit_function_float(rbnode, s):
    tmp_var = {
        'type': 'value', 
        'nodetype': 'float', 
        'value': rbnode.value, 
    }
    s['tmp_var'] = tmp_var
    return


In [ ]:
def visit_function_associative_parenthesis(rbnode, s):
    visit_function_rbnode(rbnode.value, s)

In [ ]:
def visit_function_return(rbnode, s):
    my_assert(rbnode.value.type == 'tuple', rbnode, s)
    
    returned_tuple = rbnode.value
    my_assert(len(returned_tuple.value) ==  2, rbnode, s)

    rb_period = returned_tuple.value[0]
    child_state = {
        'keyword': 'expression',
        'local_variables': s['local_variables'],
    }
    visit_function_rbnode(rb_period, child_state)
    returned_period = child_state['tmp_var']
    my_assert(returned_period['type'] == 'period', rbnode, s)

    rb_value = returned_tuple.value[1]
    child_state = {
        'keyword': 'expression',
        'local_variables': s['local_variables'],
    }
    visit_function_rbnode(rb_value, child_state)
    returned_value = child_state['tmp_var']
    my_assert(returned_value['type'] == 'value', rbnode, s)

    returned_value = {
        'type': 'return', 
        'nodetype': 'return', 
        'period': returned_period, 
        'value': returned_value,
    }
    my_assert('return' not in s, rbnode, s)
    s['return'] = returned_value
    return

    


In [ ]:
def visit_function_comparison(rbnode, s):
    my_assert(rbnode.value.type == "comparison_operator", rbnode, s)
    op = rbnode.value.first
    my_assert(not rbnode.value.second, rbnode, s)

    parsed_args = []
    for arg in [rbnode.first, rbnode.second]:
        child_state = {
            'keyword': 'expression',
            'local_variables': s['local_variables'],
        }
        visit_function_rbnode(arg, child_state)
        parsed_arg = child_state['tmp_var']
        parsed_args.append(parsed_arg)

    tmp_var = {
        'type': 'value', 
        'nodetype': 'arithmetic_operation', 
        'op': op, 
        'operands': parsed_args,
    }
    s['tmp_var'] = tmp_var
    return


In [ ]:
def visit_function_comment(rbnode, s):
    # ignored (TODO)
    return

In [ ]:
def visit_function_list(rbnode, s):
    # ignored (TODO)
    tmp_var = {
        'type': 'list', 
        'nodetype': 'list', 
        'rbnode': rbnode,
    }
    s['tmp_var'] = tmp_var
    return


In [ ]:
def visit_function_for(rbnode, s):
    # ignored (TODO)
    return

# Function parsing

In [ ]:
parsed_functions = {}

for module_name, module in parsed_classes.items():
    print('Visiting module {}'.format(module_name))
    
    parsed_functions[module_name] = {
        'classes': {},
    }
    
    for class_name, cl in module['parsed_classes'].items():
        print('Visiting class {} to parse its function(s)'.format(class_name))
        
        parsed_functions[module_name]['classes'][class_name] = {
            'parsed_functions': {},
        }
        
        for function_name, fn in cl['class_functions'].items():
            print('Visiting function {}'.format(function_name))
            
            s = {
                'keyword': 'function',
                'module_name': module_name,
                'class_name': class_name,
                'function_name': function_name,
                'local_variables': {
                    'period': {'type': 'period', 'nodetype': 'builtin-period'},
                    'simulation': {'type': 'simulation'},
                    'self': {'type': 'self'},
                },
            }
            
            for rbnode in fn['instructions']:
                visit_function_rbnode(rbnode, s)

            parsed_functions[module_name]['classes'][class_name]['parsed_functions'][function_name] = {
                'return': s['return'],
                }


In [ ]:
angry_rbnode

In [ ]:
angry_rbnode.help()

In [ ]:
angry_s

In [ ]:
parsed_classes['prelevements_obligatoires/impot_revenu/reductions_impot.py']['parsed_classes'][
    'patnat']['class_functions']['function_20100101_20101231']['instructions']